In [2]:
!pip install -q torch transformers pillow

In [ ]:
from transformers import LlavaForConditionalGeneration, AutoProcessor
import torch

model_path = "chaoyinshe/llava-med-v1.5-mistral-7b-hf"

model = LlavaForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",   # requires FA2
    device_map="auto"                          # multi-GPU ready
)

processor = AutoProcessor.from_pretrained(model_path)

In [ ]:
from PIL import Image
import requests

# loading the image i wanna use
image_url = "your_image_url_here"
image = Image.open(requests.get(image_url, stream=True).raw)

text_prompt = "What is the diagnosis in this X-ray image?"

inputs = processor(text=text_prompt, images=image, return_tensors="pt")

In [ ]:
# inference
outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
logits_per_text = outputs.logits_per_text

predicted_class = logits_per_text.argmax().item()
print(f"Predicted class index: {predicted_class}")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image)
plt.title(f"Prediction: {predicted_class}")
plt.show()